In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import torch.nn as nn
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from tqdm.notebook import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size=16


data

In [2]:
class CustomDataset(Dataset):
    def __init__(self, image,mask):
        self.img=image
        self.mask=mask
    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        image_tensor=self.img[idx]
        mask_tensor=self.mask[idx]
        return image_tensor,mask_tensor
    
image_path='../../data/1-cycle_10%_초기데이터/segmentation/image/*.tiff'
mask_path='../../data/1-cycle_10%_초기데이터/segmentation/mask/*.tiff'
train_image_list=glob(image_path)[:100]
train_mask_list=glob(mask_path)[:100]
test_image_list=glob(image_path)[100:]
test_mask_list=glob(mask_path)[100:]

train_tensor_image=torch.empty(len(train_image_list),3,512,512)
train_tensor_mask=torch.empty(len(train_mask_list),1,512,512)
tf = ToTensor()
for i in range(len(train_image_list)):
    image=tf(Image.open(train_image_list[i]))/255
    mask=tf(Image.open(train_mask_list[i]).convert("L"))/255
    train_tensor_image[i]=image
    train_tensor_mask[i]=mask
train_tensor_image1=copy(train_tensor_image)
train_tensor_mask1=copy(train_tensor_mask)

for aug in range(9):
    train_tensor_image1=torch.cat([train_tensor_image1,train_tensor_image], dim=0)
    train_tensor_mask1=torch.cat([train_tensor_mask1,train_tensor_mask], dim=0)

test_tensor_image=torch.empty(len(test_image_list),3,512,512)
test_tensor_mask=torch.empty(len(test_mask_list),1,512,512)
tf = ToTensor()
for i in range(len(test_image_list)):
    image=tf(Image.open(test_image_list[i]))/255
    mask=tf(Image.open(test_mask_list[i]).convert("L"))/255
    test_tensor_image[i]=image
    test_tensor_mask[i]=mask
test_tensor_image1=copy(test_tensor_image)
test_tensor_mask1=copy(test_tensor_mask)

for aug in range(4):
    test_tensor_image1=torch.cat([test_tensor_image1,test_tensor_image], dim=0)
    test_tensor_mask1=torch.cat([test_tensor_mask1,test_tensor_mask], dim=0)
    

train_dataset=CustomDataset(train_tensor_image1,train_tensor_mask1)
test_dataset=CustomDataset(test_tensor_image1 ,test_tensor_mask1)
dataset_size = len(test_dataset)
test_size = int(dataset_size * 0.5)
validation_size = dataset_size-test_size
validation_dataset, test_dataset = random_split(test_dataset, [validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

model

In [3]:
def convrelu(in_channels, out_channels, kernel, padding):
  return nn.Sequential(
    nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
    nn.ReLU(inplace=True),
  )


class ResNetUNet(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.base_model = torchvision.models.resnet18(pretrained=True)
    self.base_layers = list(self.base_model.children())

    self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
    self.layer0_1x1 = convrelu(64, 64, 1, 0)
    self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
    self.layer1_1x1 = convrelu(64, 64, 1, 0)
    self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
    self.layer2_1x1 = convrelu(128, 128, 1, 0)
    self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
    self.layer3_1x1 = convrelu(256, 256, 1, 0)
    self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
    self.layer4_1x1 = convrelu(512, 512, 1, 0)

    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
    self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
    self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
    self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

    self.conv_original_size0 = convrelu(3, 64, 3, 1)
    self.conv_original_size1 = convrelu(64, 64, 3, 1)
    self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

    self.conv_last = nn.Conv2d(64, n_class, 1)

  def forward(self, input):
    x_original = self.conv_original_size0(input)
    x_original = self.conv_original_size1(x_original)

    layer0 = self.layer0(input)
    layer1 = self.layer1(layer0)
    layer2 = self.layer2(layer1)
    layer3 = self.layer3(layer2)
    layer4 = self.layer4(layer3)

    layer4 = self.layer4_1x1(layer4)
    x = self.upsample(layer4)
    layer3 = self.layer3_1x1(layer3)
    x = torch.cat([x, layer3], dim=1)
    x = self.conv_up3(x)

    x = self.upsample(x)
    layer2 = self.layer2_1x1(layer2)
    x = torch.cat([x, layer2], dim=1)
    x = self.conv_up2(x)

    x = self.upsample(x)
    layer1 = self.layer1_1x1(layer1)
    x = torch.cat([x, layer1], dim=1)
    x = self.conv_up1(x)

    x = self.upsample(x)
    layer0 = self.layer0_1x1(layer0)
    x = torch.cat([x, layer0], dim=1)
    x = self.conv_up0(x)

    x = self.upsample(x)
    x = torch.cat([x, x_original], dim=1)
    x = self.conv_original_size2(x)

    out = self.conv_last(x)

    return out
model = ResNetUNet(1)
model = model.to(device)
summary(model,(batch_size,3,512,512))

/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                   Output Shape              Param #
ResNetUNet                               [16, 1, 512, 512]         513,000
├─Sequential: 1-1                        [16, 64, 512, 512]        --
│    └─Conv2d: 2-1                       [16, 64, 512, 512]        1,792
│    └─ReLU: 2-2                         [16, 64, 512, 512]        --
├─Sequential: 1-2                        [16, 64, 512, 512]        --
│    └─Conv2d: 2-3                       [16, 64, 512, 512]        36,928
│    └─ReLU: 2-4                         [16, 64, 512, 512]        --
├─Sequential: 1-3                        [16, 64, 256, 256]        --
│    └─Conv2d: 2-5                       [16, 64, 256, 256]        9,408
│    └─BatchNorm2d: 2-6                  [16, 64, 256, 256]        128
│    └─ReLU: 2-7                         [16, 64, 256, 256]        --
├─Sequential: 1-4                        [16, 64, 128, 128]        --
│    └─MaxPool2d: 2-8                    [16, 64, 128, 128]        --

In [4]:
checkpoint_path = "../../model/1-cycle.pth"

def diceloss(pred, target):
    smooth = 1.
    iflat = pred.contiguous().view(-1)
    tflat = target.contiguous().view(-1)
    intersection = (iflat * tflat).sum()
    A_sum = torch.sum(iflat * iflat)
    B_sum = torch.sum(tflat * tflat)
    return 1 - ((2. * intersection + smooth) / (A_sum + B_sum + smooth) )
def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = torch.sigmoid(pred)
    dice = diceloss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    return loss


In [5]:
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
for l in model.base_layers:
      for param in l.parameters():
        param.requires_grad = False

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
metrics = defaultdict(float)
for epoch in range(10):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    for x, y in train:
        model.train()
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = calc_loss(predict, y,metrics) # cost 구함
        acc=1-calc_loss(predict, y,metrics)
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{10} Step: {count+1} dice_loss : {running_loss/count:.4f} dice_score: {acc_loss/count:.4f}")
        train_loss_list.append((running_loss/count))
        train_acc_list.append((acc_loss/count).cpu().detach().numpy())
#validation
    val=tqdm(validation_dataloader)
    model.eval()
    count=0
    val_running_loss=0.0
    acc_loss=0
    with torch.no_grad():
        for x, y in val:
            y = y.to(device).float()
            count+=1
            x=x.to(device).float()
            
            predict = model(x).to(device)
            cost = calc_loss(predict, y,metrics) # cost 구함
            acc=1-calc_loss(predict, y,metrics)
            val_running_loss+=cost.item()
            acc_loss+=acc
            val.set_description(f"Validation epoch: {epoch+1}/{10} Step: {count+1} dice_loss : {val_running_loss/count:.4f}  dice_score: {acc_loss/count:.4f}")
            val_loss_list.append((val_running_loss/count))
            val_acc_list.append((acc_loss/count).cpu().detach().numpy())
with torch.no_grad():
    count=0
    test_running_loss=0
    acc_loss=0
    for x, y in test_dataloader:
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        
        predict = model(x).to(device)
        cost = calc_loss(predict, y,metrics) # cost 구함
        acc=1-calc_loss(predict, y,metrics)
        test_running_loss+=cost.item()
        acc_loss+=acc
print('test prediction dice score: %f'%(acc_loss/count).item())

test prediction dice score: 0.688202
